# Long-term Cognitive Networks for pattern classification

Long-term Cognitive Networks are trained with an inverse learning rule. In this model, the weights connecting the input neurons are coefficients of multiple regressions models while the weights connecting the temporal states with and outputs are computed using a learning method (the Moore–Penrose inverse method when no regularization is needed or the Ridge regression method when the model might overfit the data).

The syntax for the use of the LTCN classifier is compatible with scikit-learn library. 

In [1]:
!pip install ltcn

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import re
import numpy as np
import random
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from ltcn.LTCN import LTCN

from fitter import Fitter, get_common_distributions, get_distributions
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.datasets import make_friedman3, make_s_curve
from sklearn.linear_model import BayesianRidge, ElasticNet
from sklearn.model_selection import KFold, train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.neural_network import MLPClassifier

!pip install imbalanced-learn
!pip install lightgbm
!pip install wittgenstein --user
!pip install explainerdashboard --user

from imblearn.over_sampling import ADASYN
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer

# for Box-Cox Transformation
from scipy import stats

# plotting modules
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.express as px

from tqdm import tqdm
import math

from sklearn.datasets import make_blobs
from matplotlib import pyplot
from pandas import DataFrame

#Multilabel Stratified K Fold Creation
!pip install iterative-stratification
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
#Collect data from sensors (i.e., time-series data) installed in the manufacturing system 
ai4i2020_encoded_balanced=pd.read_csv(r'..\\..\\dataset\\raw_data\\ai4i2020_encoded_balanced.csv')
ai4i2020_encoded_balanced.head()

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,0,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,0,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,0,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,0,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


In [3]:
# Run in local
# Fold index
fold = 1

original_X_train = pd.read_csv('..\\..\\dataset\\k-fold cross validation datasets\\' + str(fold) + "\\Training Dataset" + "\\X_train_iter_" + str(fold) + ".csv")
original_X_test = pd.read_csv('..\\..\\dataset\\k-fold cross validation datasets\\' + str(fold) + "\\Test Dataset" + "\\X_test_iter_" + str(fold) + ".csv")

X_train = pd.read_csv('..\\..\\dataset\\k-fold cross validation datasets\\' + str(fold) + "\\Training Dataset" + "\\Scaled_X_train_iter_" + str(fold) + ".csv", names=['Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]'])
y_train = pd.read_csv('..\\..\\dataset\\k-fold cross validation datasets\\' + str(fold) + "\\Training Dataset" + "\\y_train_iter_" + str(fold) + ".csv")
X_test = pd.read_csv('..\\..\\dataset\\k-fold cross validation datasets\\' + str(fold) + "\\Test Dataset" + "\\Scaled_X_test_iter_" + str(fold) + ".csv", names=['Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]'])
y_test = pd.read_csv('..\\..\\dataset\\k-fold cross validation datasets\\' + str(fold) + "\\Test Dataset" + "\\y_test_iter_" + str(fold) + ".csv")

In [4]:
ai4i2020_encoded_balanced = ai4i2020_encoded_balanced.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
original_X_train = original_X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
original_X_test = original_X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_train = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_test = X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
y_train = y_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
y_test = y_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [5]:
acc_list = []
auc_list = []
kappa_list = []

In [6]:
def reset_random_seeds():
    os.environ['PYTHONHASHSEED']=str(42)
    np.random.seed(42)
    random.seed(42)

def kappa_scorer(Y_pred, y_test_numpy):
    return cohen_kappa_score(y_test_numpy,Y_pred.round())


In [7]:
labels = y_train # here we have the classes
n_features = len(X_train.columns)+len(y_train.columns) #number of features
n_classes = len(np.unique(y_train)) #number of unique classes

Long-term Cognitive Network classifier

"""

        Parameters
        ----------
            T         :  {int}, default 20
                         Number of iterations to be performed
            phi       :  {float}, default 0.8
                         Amount of non-linearity during reasoning.
            method    :  {String}, default 'inverse'
                         Regression approach ('inverse', 'ridge')
            function  :  {String}, default 'sigmoid'
                         Activation function ('sigmoid', 'tanh')
            alpha :      {float}, default 1.0E-4
                         Positive penalization for L2-regularization

        """

In [8]:
from sklearn.model_selection import ParameterGrid
from tqdm.notebook import trange, tqdm

X_train_numpy = X_train.to_numpy()
y_train_numpy = y_train.to_numpy()
X_test_numpy = X_test.to_numpy()
y_test_numpy = y_test.to_numpy()


acc_list = []
auc_list = []
kappa_list = []
twf_list = []
hdf_list = []
pwf_list = []
osf_list = []
av_succ = []
comb_list = []

# hyper-parameter tuning using grid search happens here!!
param_grid = {'method': ['inverse'], 'function': ['sigmoid', 'tanh'], 'phi': np.arange(0.5, 1.0, 0.1), 'T': [5, 10, 15], 'alpha': [0, 1.0E-2, 1.0E+2]}

grid = ParameterGrid(param_grid)

for comb in tqdm(range(0,len(grid)), desc='Combinations'):
    
    #Build the LTCN Classifier
    model = LTCN(method='inverse')
    #display("Combination " + str(comb))
    model.__init__(**grid[comb])
    model.fit(X_train_numpy,y_train_numpy)
    
    #Predictions for the test set
    y_test_pred_ltcn = model.predict(X_test_numpy)
    y_test_pred_ltcn_final = y_test_pred_ltcn.round()
    
    #Evaluation metrics for the test dataset
    accuracy_ltcn = accuracy_score(y_test_numpy, y_test_pred_ltcn.round())
    cohen_score_ltcn = cohen_kappa_score(y_test_numpy, y_test_pred_ltcn.round())
    auc_ltcn = roc_auc_score(y_test_numpy, y_test_pred_ltcn.round())
    
    #LTCN reasoning process
    # Z : Matrix with the neurons' last activation values.
    A = X_test_numpy
    A0 = A
    Z = A0

    for t in range(model.T):
        A = model.phi * model.transform(np.matmul(A, model.W1)) + (1 - model.phi) * A0
        Z = np.concatenate((Z, A), axis=1)
    local_feature_importance_ltcn = pd.DataFrame(A, columns=['Type', 'AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin'])
    
    #Find the observations' indexes that the model correctly predicted as faulty
    true_positive_indexes = []
    i=1;

    #According to the chosen model
    #For example, y_test_pred_lgbmc for LGBM Classifier
    for k in range(0,len(y_test)):
        if (y_test.loc[k,'Machinefailure']==int(y_test_pred_ltcn_final[k])) and (y_test.loc[k,'Machinefailure']==1):
            true_positive_indexes.append(k)
            i=i+1;

    #Find the observations that the selected model correctly predicted as faulty based on the previous indexes
    true_positive_observations = pd.DataFrame()
    for k in range(0,len(true_positive_indexes)):
        new_row = pd.concat([original_X_test.loc[true_positive_indexes[k],:],y_test.loc[true_positive_indexes[k],:]], axis=0)
        true_positive_observations = true_positive_observations.append(new_row, ignore_index=True)
    
    if not true_positive_observations.empty:
        
        
        #Append in lists
        acc_list.append(accuracy_ltcn)
        kappa_list.append(cohen_score_ltcn)
        auc_list.append(auc_ltcn)
        
    
        true_positive_observations_with_failure_modes = pd.DataFrame()
        true_positive_observations_with_failure_modes = ai4i2020_encoded_balanced.join(true_positive_observations.set_index(['Type', 'AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin', 'Machinefailure']), ['Type', 'AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin', 'Machinefailure'], how='right')       
        final_true_positive_observations_with_failure_modes = true_positive_observations_with_failure_modes.reset_index(drop=True)
    
        #display("Number of true positive predictions: " + str(len(final_true_positive_observations_with_failure_modes)))

        number_of_TWF_failures_in_true_positive_predictions = 0
        number_of_HDF_failures_in_true_positive_predictions = 0
        number_of_PWF_failures_in_true_positive_predictions = 0
        number_of_OSF_failures_in_true_positive_predictions = 0
        number_of_random_failures_in_true_positive_predictions=0

        for k in range(0,len(final_true_positive_observations_with_failure_modes)):
            if final_true_positive_observations_with_failure_modes.loc[k,"TWF"]==1: #if the failure mode is TWF
                number_of_TWF_failures_in_true_positive_predictions = number_of_TWF_failures_in_true_positive_predictions + 1;
            if final_true_positive_observations_with_failure_modes.loc[k,"HDF"]==1: #if the failure mode is HDF
                number_of_HDF_failures_in_true_positive_predictions = number_of_HDF_failures_in_true_positive_predictions + 1;
            if final_true_positive_observations_with_failure_modes.loc[k,"PWF"]==1: #if the failure mode is PWF
                number_of_PWF_failures_in_true_positive_predictions = number_of_PWF_failures_in_true_positive_predictions + 1;
            if final_true_positive_observations_with_failure_modes.loc[k,"OSF"]==1: #if the failure mode is OSF
                number_of_OSF_failures_in_true_positive_predictions = number_of_OSF_failures_in_true_positive_predictions + 1;
            if (final_true_positive_observations_with_failure_modes.loc[k,"Machinefailure"]==1) and (final_true_positive_observations_with_failure_modes.loc[k,"TWF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"HDF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"PWF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"OSF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"RNF"]==0):
                number_of_random_failures_in_true_positive_predictions = number_of_random_failures_in_true_positive_predictions+1;
    
        #Calculate the correct explanations
        correct_explanations = 0
        correct_explanations_TWF = 0
        correct_explanations_HDF = 0
        correct_explanations_PWF = 0
        correct_explanations_OSF = 0

        for k in range(0,len(true_positive_observations)):
            #if the failure mode is TWF
            if final_true_positive_observations_with_failure_modes.loc[k,"TWF"]==1:
                #if tool wear is the most important feature
                if local_feature_importance_ltcn.loc[true_positive_indexes[k],"Toolwearmin"] == max(local_feature_importance_ltcn.loc[true_positive_indexes[k],:]):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_TWF = correct_explanations_TWF + 1;
                
            #if the failure mode is HDF
            if final_true_positive_observations_with_failure_modes.loc[k,"HDF"]==1: 
                maximum1 = max(local_feature_importance_ltcn.loc[true_positive_indexes[k],:]) 
                maximum2 = max(local_feature_importance_ltcn.loc[true_positive_indexes[k],:], key = lambda x: min(local_feature_importance_ltcn.loc[true_positive_indexes[k],:])-1 if (x == maximum1) else x)
                if (local_feature_importance_ltcn.loc[true_positive_indexes[k],"AirtemperatureK"] == maximum1) or (local_feature_importance_ltcn.loc[true_positive_indexes[k],"AirtemperatureK"] == maximum2) or (local_feature_importance_ltcn.loc[true_positive_indexes[k],"ProcesstemperatureK"] == maximum1) or (local_feature_importance_ltcn.loc[true_positive_indexes[k],"ProcesstemperatureK"] == maximum2):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_HDF = correct_explanations_HDF + 1;
            
            
            #if the failure mode is PWF
            if final_true_positive_observations_with_failure_modes.loc[k,"PWF"]==1: 
                maximum1 = max(local_feature_importance_ltcn.loc[true_positive_indexes[k],:]) 
                maximum2 = max(local_feature_importance_ltcn.loc[true_positive_indexes[k],:], key = lambda x: min(local_feature_importance_ltcn.loc[true_positive_indexes[k],:])-1 if (x == maximum1) else x)
                if (local_feature_importance_ltcn.loc[true_positive_indexes[k],"Rotationalspeedrpm"] == maximum1) or (local_feature_importance_ltcn.loc[true_positive_indexes[k],"Rotationalspeedrpm"] == maximum2) or (local_feature_importance_ltcn.loc[true_positive_indexes[k],"TorqueNm"] == maximum1) or (local_feature_importance_ltcn.loc[true_positive_indexes[k],"TorqueNm"] == maximum2):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_PWF = correct_explanations_PWF + 1;
            
            #if the failure mode is OSF        
            if final_true_positive_observations_with_failure_modes.loc[k,"OSF"]==1:
                maximum1 = max(local_feature_importance_ltcn.loc[true_positive_indexes[k],:]) 
                maximum2 = max(local_feature_importance_ltcn.loc[true_positive_indexes[k],:], key = lambda x: min(local_feature_importance_ltcn.loc[true_positive_indexes[k],:])-1 if (x == maximum1) else x)
                if (local_feature_importance_ltcn.loc[true_positive_indexes[k],"Toolwearmin"] == maximum1) or (local_feature_importance_ltcn.loc[true_positive_indexes[k],"Toolwearmin"] == maximum2) or (local_feature_importance_ltcn.loc[true_positive_indexes[k],"TorqueNm"] == maximum1) or (local_feature_importance_ltcn.loc[true_positive_indexes[k],"TorqueNm"] == maximum2):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_OSF = correct_explanations_OSF + 1;
        
        denominator = 0
        
        if number_of_TWF_failures_in_true_positive_predictions != 0:
            twf_list.append(correct_explanations_TWF/number_of_TWF_failures_in_true_positive_predictions)
            twf_success = correct_explanations_TWF/number_of_TWF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            twf_list.append(correct_explanations_TWF)
            twf_success = 0
        if number_of_HDF_failures_in_true_positive_predictions != 0:
            hdf_list.append(correct_explanations_HDF/number_of_HDF_failures_in_true_positive_predictions)
            hdf_success = correct_explanations_HDF/number_of_HDF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            hdf_list.append(correct_explanations_HDF)
            hdf_success = 0
        if number_of_PWF_failures_in_true_positive_predictions != 0:
            pwf_list.append(correct_explanations_PWF/number_of_PWF_failures_in_true_positive_predictions)
            pwf_success = correct_explanations_PWF/number_of_PWF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            pwf_list.append(correct_explanations_PWF)
            pwf_success = 0
        if number_of_OSF_failures_in_true_positive_predictions != 0:
            osf_list.append(correct_explanations_OSF/number_of_OSF_failures_in_true_positive_predictions)
            osf_success = correct_explanations_OSF/number_of_OSF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            osf_list.append(correct_explanations_OSF)
            osf_success = 0
    
        average_success = (twf_success+hdf_success+pwf_success+osf_success)/denominator
        av_succ.append(average_success)
        comb_list.append(comb)
        
    
result_df = pd.DataFrame({'Accuracy': acc_list, 'AUC': auc_list, 'Kappa':kappa_list, 'TWF':twf_list, 'HDF':hdf_list, 'PWF':pwf_list, 'OSF':osf_list, 'Average Success':av_succ, 'Combinations':comb_list})
final_df = result_df.sort_values(by=['AUC'], ascending=False)
final_df = final_df.reset_index(drop=True)
best_estimator_params = grid[final_df.loc[0,"Combinations"]]
display(best_estimator_params)

Combinations:   0%|          | 0/90 [00:00<?, ?it/s]

{'phi': 0.8999999999999999,
 'method': 'inverse',
 'function': 'tanh',
 'alpha': 100.0,
 'T': 15}

In [9]:
final_df

,Accuracy,AUC,Kappa,TWF,HDF,PWF,OSF,Average Success,Combinations
0,0.958881,0.957892,0.917436,0.319444,1.000000,0.851852,0.841270,0.753142,89
1,0.958881,0.957892,0.917436,0.319444,1.000000,0.851852,0.841270,0.753142,79
2,0.958881,0.957892,0.917436,0.319444,1.000000,0.851852,0.841270,0.753142,69
3,0.957738,0.956695,0.915134,0.394366,0.996078,0.873134,0.869048,0.783157,77
4,0.957738,0.956695,0.915134,0.394366,0.996078,0.873134,0.869048,0.783157,67
...,...,...,...,...,...,...,...,...,...
85,0.920617,0.919520,0.840598,0.500000,0.996063,0.915966,0.896825,0.827214,25
86,0.920617,0.919520,0.840598,0.500000,0.996063,0.915966,0.896825,0.827214,15
87,0.916619,0.915116,0.832437,0.254902,1.000000,0.841270,0.853175,0.737337,29
88,0.916619,0.915116,0.832437,0.254902,1.000000,0.841270,0.853175,0.737337,19


## Calculate the evaluation metrics for all folds of the best estimator 

In [10]:
acc_folds_best_estimator = []
auc_folds_best_estimator = []
kappa_folds_best_estimator = []
twf_folds_best_estimator = []
hdf_folds_best_estimator = []
pwf_folds_best_estimator = []
osf_folds_best_estimator = []
av_folds_best_estimator = []
fold_list = []


for fold in tqdm(range(1,11), desc='Folds'):
    
    #Dataset for each fold
    original_X_train = pd.read_csv('..\\..\\dataset\\k-fold cross validation datasets\\' + str(fold) + "\\Training Dataset" + "\\X_train_iter_" + str(fold) + ".csv")
    original_X_test = pd.read_csv('G:\\.shortcut-targets-by-id\\1-wapAl6N5YrCs68c4NiFKyvybXTXmdgZ\\Ph_D_Tyrovolas\\Our Papers\\3rd_Paper-Proposal\\Testbed Codes\\AI4I_Case_Study\\k-fold cross validation datasets\\' + str(fold) + "\\Test Dataset" + "\\X_test_iter_" + str(fold) + ".csv")

    X_train = pd.read_csv('..\\..\\dataset\\k-fold cross validation datasets\\' + str(fold) + "\\Training Dataset" + "\\Scaled_X_train_iter_" + str(fold) + ".csv", names=['Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]'])
    y_train = pd.read_csv('..\\..\\dataset\\k-fold cross validation datasets\\' + str(fold) + "\\Training Dataset" + "\\y_train_iter_" + str(fold) + ".csv")
    X_test = pd.read_csv('..\\..\\dataset\\k-fold cross validation datasets\\' + str(fold) + "\\Test Dataset" + "\\Scaled_X_test_iter_" + str(fold) + ".csv", names=['Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]'])
    y_test = pd.read_csv('..\\..\\dataset\\k-fold cross validation datasets\\' + str(fold) + "\\Test Dataset" + "\\y_test_iter_" + str(fold) + ".csv")
    
    original_X_train = original_X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    original_X_test = original_X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    X_train = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    X_test = X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    y_train = y_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    y_test = y_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    
    #Dataframes to numpy
    X_train_numpy = X_train.to_numpy()
    y_train_numpy = y_train.to_numpy()
    X_test_numpy = X_test.to_numpy()
    y_test_numpy = y_test.to_numpy()
    
    
    #Build the LTCN Classifier
    model = LTCN(method='inverse')
    #display("Combination " + str(comb))
    model.__init__(**grid[final_df.loc[0,"Combinations"]])
    model.fit(X_train_numpy,y_train_numpy)
    
    #Predictions for the test set
    y_test_pred_ltcn = model.predict(X_test_numpy)
    y_test_pred_ltcn_final = y_test_pred_ltcn.round()
    
    #Evaluation metrics for the test dataset
    accuracy_ltcn = accuracy_score(y_test_numpy, y_test_pred_ltcn.round())
    cohen_score_ltcn = cohen_kappa_score(y_test_numpy, y_test_pred_ltcn.round())
    auc_ltcn = roc_auc_score(y_test_numpy, y_test_pred_ltcn.round())
    
    #LTCN reasoning process
    # Z : Matrix with the neurons' last activation values.
    A = X_test_numpy
    A0 = A
    Z = A0

    for t in range(model.T):
        A = model.phi * model.transform(np.matmul(A, model.W1)) + (1 - model.phi) * A0
        Z = np.concatenate((Z, A), axis=1)
    local_feature_importance_ltcn = pd.DataFrame(A, columns=['Type', 'AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin'])
    
    
    R = model.W2[0:len(Z[0]),:]
    Q = model.W2[len(Z[0]),:]

    R = R.reshape(-1,6)

    relevance_score = []
    feature_names = X_train.columns.to_list()

    for feature in range(0,len(model.W1)):
        score = sum(abs(model.W1[feature,:])) + sum(abs(R[:,feature]))
        relevance_score.append(score)
    
    glob_feat_import_ltcn = pd.DataFrame({'Feature': feature_names, 'Relevance': relevance_score})
    glob_feat_import_ltcn = glob_feat_import_ltcn.sort_values(by='Relevance', ascending=False)
    display(glob_feat_import_ltcn)
    
    #Find the observations' indexes that the model correctly predicted as faulty
    true_positive_indexes = []
    i=1;

    #According to the chosen model
    #For example, y_test_pred_lgbmc for LGBM Classifier
    for k in range(0,len(y_test)):
        if (y_test.loc[k,'Machinefailure']==int(y_test_pred_ltcn_final[k])) and (y_test.loc[k,'Machinefailure']==1):
            true_positive_indexes.append(k)
            i=i+1;

    #Find the observations that the selected model correctly predicted as faulty based on the previous indexes
    true_positive_observations = pd.DataFrame()
    for k in range(0,len(true_positive_indexes)):
        new_row = pd.concat([original_X_test.loc[true_positive_indexes[k],:],y_test.loc[true_positive_indexes[k],:]], axis=0)
        true_positive_observations = true_positive_observations.append(new_row, ignore_index=True)
    
    if not true_positive_observations.empty:
        
        #Append in lists
        acc_folds_best_estimator.append(accuracy_ltcn)
        kappa_folds_best_estimator.append(cohen_score_ltcn)
        auc_folds_best_estimator.append(auc_ltcn)
        
    
        true_positive_observations_with_failure_modes = pd.DataFrame()
        true_positive_observations_with_failure_modes = ai4i2020_encoded_balanced.join(true_positive_observations.set_index(['Type', 'AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin', 'Machinefailure']), ['Type', 'AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin', 'Machinefailure'], how='right')       
        final_true_positive_observations_with_failure_modes = true_positive_observations_with_failure_modes.reset_index(drop=True)

    
        #display("Number of true positive predictions: " + str(len(final_true_positive_observations_with_failure_modes)))

        number_of_TWF_failures_in_true_positive_predictions = 0
        number_of_HDF_failures_in_true_positive_predictions = 0
        number_of_PWF_failures_in_true_positive_predictions = 0
        number_of_OSF_failures_in_true_positive_predictions = 0
        number_of_random_failures_in_true_positive_predictions=0

        for k in range(0,len(final_true_positive_observations_with_failure_modes)):
            if final_true_positive_observations_with_failure_modes.loc[k,"TWF"]==1: #if the failure mode is TWF
                number_of_TWF_failures_in_true_positive_predictions = number_of_TWF_failures_in_true_positive_predictions + 1;
            if final_true_positive_observations_with_failure_modes.loc[k,"HDF"]==1: #if the failure mode is HDF
                number_of_HDF_failures_in_true_positive_predictions = number_of_HDF_failures_in_true_positive_predictions + 1;
            if final_true_positive_observations_with_failure_modes.loc[k,"PWF"]==1: #if the failure mode is PWF
                number_of_PWF_failures_in_true_positive_predictions = number_of_PWF_failures_in_true_positive_predictions + 1;
            if final_true_positive_observations_with_failure_modes.loc[k,"OSF"]==1: #if the failure mode is OSF
                number_of_OSF_failures_in_true_positive_predictions = number_of_OSF_failures_in_true_positive_predictions + 1;
            if (final_true_positive_observations_with_failure_modes.loc[k,"Machinefailure"]==1) and (final_true_positive_observations_with_failure_modes.loc[k,"TWF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"HDF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"PWF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"OSF"]==0) and (final_true_positive_observations_with_failure_modes.loc[k,"RNF"]==0):
                number_of_random_failures_in_true_positive_predictions = number_of_random_failures_in_true_positive_predictions+1;
    
        #Calculate the correct explanations
        correct_explanations = 0
        correct_explanations_TWF = 0
        correct_explanations_HDF = 0
        correct_explanations_PWF = 0
        correct_explanations_OSF = 0

        for k in range(0,len(true_positive_observations)):
            #if the failure mode is TWF
            if final_true_positive_observations_with_failure_modes.loc[k,"TWF"]==1:
                #if tool wear is the most important feature
                if local_feature_importance_ltcn.loc[true_positive_indexes[k],"Toolwearmin"] == max(local_feature_importance_ltcn.loc[true_positive_indexes[k],:]):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_TWF = correct_explanations_TWF + 1;
                
            #if the failure mode is HDF
            if final_true_positive_observations_with_failure_modes.loc[k,"HDF"]==1: 
                maximum1 = max(local_feature_importance_ltcn.loc[true_positive_indexes[k],:]) 
                maximum2 = max(local_feature_importance_ltcn.loc[true_positive_indexes[k],:], key = lambda x: min(local_feature_importance_ltcn.loc[true_positive_indexes[k],:])-1 if (x == maximum1) else x)
                if (local_feature_importance_ltcn.loc[true_positive_indexes[k],"AirtemperatureK"] == maximum1) or (local_feature_importance_ltcn.loc[true_positive_indexes[k],"AirtemperatureK"] == maximum2) or (local_feature_importance_ltcn.loc[true_positive_indexes[k],"ProcesstemperatureK"] == maximum1) or (local_feature_importance_ltcn.loc[true_positive_indexes[k],"ProcesstemperatureK"] == maximum2):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_HDF = correct_explanations_HDF + 1;
            
            
            #if the failure mode is PWF
            if final_true_positive_observations_with_failure_modes.loc[k,"PWF"]==1: 
                maximum1 = max(local_feature_importance_ltcn.loc[true_positive_indexes[k],:]) 
                maximum2 = max(local_feature_importance_ltcn.loc[true_positive_indexes[k],:], key = lambda x: min(local_feature_importance_ltcn.loc[true_positive_indexes[k],:])-1 if (x == maximum1) else x)
                if (local_feature_importance_ltcn.loc[true_positive_indexes[k],"Rotationalspeedrpm"] == maximum1) or (local_feature_importance_ltcn.loc[true_positive_indexes[k],"Rotationalspeedrpm"] == maximum2) or (local_feature_importance_ltcn.loc[true_positive_indexes[k],"TorqueNm"] == maximum1) or (local_feature_importance_ltcn.loc[true_positive_indexes[k],"TorqueNm"] == maximum2):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_PWF = correct_explanations_PWF + 1;
            
            #if the failure mode is OSF        
            if final_true_positive_observations_with_failure_modes.loc[k,"OSF"]==1:
                maximum1 = max(local_feature_importance_ltcn.loc[true_positive_indexes[k],:]) 
                maximum2 = max(local_feature_importance_ltcn.loc[true_positive_indexes[k],:], key = lambda x: min(local_feature_importance_ltcn.loc[true_positive_indexes[k],:])-1 if (x == maximum1) else x)
                if (local_feature_importance_ltcn.loc[true_positive_indexes[k],"Toolwearmin"] == maximum1) or (local_feature_importance_ltcn.loc[true_positive_indexes[k],"Toolwearmin"] == maximum2) or (local_feature_importance_ltcn.loc[true_positive_indexes[k],"TorqueNm"] == maximum1) or (local_feature_importance_ltcn.loc[true_positive_indexes[k],"TorqueNm"] == maximum2):
                    correct_explanations = correct_explanations+1;
                    correct_explanations_OSF = correct_explanations_OSF + 1;
        
        denominator = 0
        
        if number_of_TWF_failures_in_true_positive_predictions != 0:
            twf_folds_best_estimator.append(correct_explanations_TWF/number_of_TWF_failures_in_true_positive_predictions)
            twf_success = correct_explanations_TWF/number_of_TWF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            twf_folds_best_estimator.append(correct_explanations_TWF)
            twf_success = 0
        if number_of_HDF_failures_in_true_positive_predictions != 0:
            hdf_folds_best_estimator.append(correct_explanations_HDF/number_of_HDF_failures_in_true_positive_predictions)
            hdf_success = correct_explanations_HDF/number_of_HDF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            hdf_folds_best_estimator.append(correct_explanations_HDF)
            hdf_success = 0
        if number_of_PWF_failures_in_true_positive_predictions != 0:
            pwf_folds_best_estimator.append(correct_explanations_PWF/number_of_PWF_failures_in_true_positive_predictions)
            pwf_success = correct_explanations_PWF/number_of_PWF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            pwf_folds_best_estimator.append(correct_explanations_PWF)
            pwf_success = 0
        if number_of_OSF_failures_in_true_positive_predictions != 0:
            osf_folds_best_estimator.append(correct_explanations_OSF/number_of_OSF_failures_in_true_positive_predictions)
            osf_success = correct_explanations_OSF/number_of_OSF_failures_in_true_positive_predictions
            denominator = denominator + 1
        else:
            osf_folds_best_estimator.append(correct_explanations_OSF)
            osf_success = 0
    
        average_success = (twf_success+hdf_success+pwf_success+osf_success)/denominator
        av_folds_best_estimator.append(average_success)
        fold_list.append(fold)

result_df_best_estimator = pd.DataFrame({'Accuracy': acc_folds_best_estimator, 'AUC': auc_folds_best_estimator, 'Kappa':kappa_folds_best_estimator, 'TWF':twf_folds_best_estimator, 'HDF':hdf_folds_best_estimator, 'PWF':pwf_folds_best_estimator, 'OSF':osf_folds_best_estimator, 'Average Success':av_folds_best_estimator, 'Folds':fold_list})


Folds:   0%|          | 0/10 [00:00<?, ?it/s]

,Feature,Relevance
4,TorqueNm,2.940645e+13
3,Rotationalspeedrpm,2.206178e+13
5,Toolwearmin,1.336596e+13
0,Type,1.265287e+13
1,AirtemperatureK,6.537390e+12
2,ProcesstemperatureK,6.025433e+12


,Feature,Relevance
4,TorqueNm,2.150578e+13
3,Rotationalspeedrpm,1.781522e+13
2,ProcesstemperatureK,1.636717e+13
1,AirtemperatureK,1.554237e+13
0,Type,1.422937e+13
5,Toolwearmin,4.672636e+12


,Feature,Relevance
4,TorqueNm,2.522461e+13
3,Rotationalspeedrpm,1.939218e+13
0,Type,1.447180e+13
1,AirtemperatureK,1.022223e+13
2,ProcesstemperatureK,9.459732e+12
5,Toolwearmin,9.286629e+12


,Feature,Relevance
4,TorqueNm,2.635435e+13
3,Rotationalspeedrpm,1.971738e+13
0,Type,1.311426e+13
5,Toolwearmin,1.248989e+13
1,AirtemperatureK,6.486435e+12
2,ProcesstemperatureK,5.508664e+12


...

,Feature,Relevance
4,TorqueNm,2.524654e+13
3,Rotationalspeedrpm,1.904152e+13
5,Toolwearmin,1.374193e+13
0,Type,1.305004e+13
1,AirtemperatureK,4.138978e+12
2,ProcesstemperatureK,2.574481e+12


,Feature,Relevance
4,TorqueNm,2.536941e+13
3,Rotationalspeedrpm,1.914985e+13
0,Type,1.244873e+13
5,Toolwearmin,1.195445e+13
1,AirtemperatureK,6.004029e+12
2,ProcesstemperatureK,5.774259e+12


,Feature,Relevance
4,TorqueNm,2.764048e+13
3,Rotationalspeedrpm,2.076314e+13
0,Type,1.615289e+13
5,Toolwearmin,9.895899e+12
1,AirtemperatureK,9.802970e+12
2,ProcesstemperatureK,9.687180e+12


,Feature,Relevance
5,Toolwearmin,1.882359e+13
4,TorqueNm,1.684078e+13
3,Rotationalspeedrpm,1.553148e+13
0,Type,8.845621e+12
1,AirtemperatureK,4.292343e+12
2,ProcesstemperatureK,1.799020e+12


,Feature,Relevance
4,TorqueNm,2.718386e+13
3,Rotationalspeedrpm,2.062438e+13
0,Type,1.237108e+13
5,Toolwearmin,1.140694e+13
1,AirtemperatureK,6.523633e+12
2,ProcesstemperatureK,5.717134e+12


In [11]:
result_df_best_estimator

,Accuracy,AUC,Kappa,TWF,HDF,PWF,OSF,Average Success,Folds
0,0.958881,0.957892,0.917436,0.319444,1.0,0.851852,0.841270,0.753142,1
1,0.945174,0.943627,0.889798,0.394366,1.0,0.840909,0.816514,0.762947,2
2,0.943461,0.942625,0.886517,0.262295,1.0,0.848837,0.822222,0.733339,3
3,0.956596,0.955232,0.912785,0.283784,1.0,0.906667,0.855372,0.761456,4
4,0.945174,0.943998,0.889880,0.341772,1.0,0.766423,0.813043,0.730310,5
5,0.955454,0.954458,0.910556,0.295082,1.0,0.837838,0.810811,0.735933,6
6,0.953170,0.952010,0.905939,0.314286,1.0,0.850299,0.833333,0.749480,7
7,0.946857,0.945725,0.893274,0.307692,1.0,0.773256,0.809955,0.722726,8
8,0.952571,0.952749,0.904870,0.393939,1.0,0.840764,0.815574,0.762569,9
9,0.956571,0.955241,0.912736,0.270270,1.0,0.807229,0.812500,0.722500,10
